In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

# Spapros Selection: Create a probeset with spapros

This tutorial shows how to select a gene probeset with spapros.

The used dataset contains 3k PBMCs from a healthy donor and is publicly available from
[10x Genomics](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k)
[here](http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz).
It is also available via scanpy [here](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.pbmc3k.html#scanpy.datasets.pbmc3k).

You can also derive it from scanpy [here](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.pbmc3k.html#scanpy.datasets.pbmc3k), like we will do in this tutorial.

The preprocessed Anndata consists of log-normalized single cell RNAseq counts of 2638 cells for 13714 genes.

## Import packages and setup

To run the notebook locally, create a conda environment using this [environment.yaml](TODO link):

    conda create -f environment.yaml

Then add the conda environment as ipython kernel:

    python -m ipykernel install --user --name spapros --display-name Python (spapros)



In [2]:
import spapros
from spapros import se, pl
import scanpy as sc

In [3]:
sc.settings.verbosity = 1
sc.logging.print_header()
print(f"spapros=={spapros.__version__}")

scanpy==1.8.2 anndata==0.7.8 umap==0.5.2 numpy==1.21.4 scipy==1.7.2 pandas==1.3.4 scikit-learn==1.0.1 statsmodels==0.13.1 python-igraph==0.9.8 pynndescent==0.5.5
spapros==0.1.0


## Preprocess data

The spapros pipeline reaches the best results, if the data is logarithmized and normalized.
We are using these preprocessing steps from the [scanpy pipeline](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html) but only save the highly variable genes and the size factors to keep the adata clean.

In [4]:
adata = sc.datasets.pbmc3k()
adata_tmp = sc.datasets.pbmc3k_processed()
adata = adata[adata_tmp.obs_names,adata_tmp.var_names]
adata_raw = adata.copy()
sc.pp.normalize_total(adata,target_sum=1e4,key_added="size_factors")
sc.pp.highly_variable_genes(adata,flavor="cell_ranger",n_top_genes=1000)
adata.X = adata_raw.X
sc.pp.log1p(adata)
adata.obs['celltype'] = adata_tmp.obs['louvain']
adata

AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'size_factors', 'celltype'
    var: 'gene_ids', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'

## Start the selection

TODO short description of spapros selection pipeline with figure

In [5]:
# create an instance of the ProbesetSelector class
selector = se.ProbesetSelector(adata, n=50, celltype_key="celltype", verbosity=1, save_dir=None)

The following celltypes' test set sizes for forest training are below min_test_n (=20):
	 Dendritic cells : 9
	 Megakaryocytes  : 3


In [6]:
# invoke the central method
selector.select_probeset()


Output()

Select pca genes...
Select genes based on differential expression and forests as baseline for the final forests...


Train hierarchical trees:   0%|          | 0/3 [00:00<?, ?it/s]

Add DE genes with specific reference groups to improve tree performance...


Train hierarchical trees:   0%|          | 0/3 [00:00<?, ?it/s]

...

Train hierarchical trees:   0%|          | 0/3 [00:00<?, ?it/s]

Train hierarchical trees:   0%|          | 0/3 [00:00<?, ?it/s]

Train final forests by adding genes from the DE_baseline forest for celltypes with low performance...


Train hierarchical trees:   0%|          | 0/3 [00:00<?, ?it/s]

AssertionError: 

The selected probeset can be examined like this:

In [ ]:
selected_probeset = selector.probeset
selected_probeset

In [ ]:
# list of the selected genes:
selected_probeset.index[selected_probeset.selection]

## Vizualize the results

In [ ]:
pl.masked_dotplot(adata, selector)

## What's next?

Now you should evaluate the selected probeset. See our [basic evaluation tutoral](https://spapros.readthedocs.io/en/latest/tutorials/spapros_tutorial_basic_evaluation.html).

If you want to customize and improve the probeset selection, have a look at the [advanced selection tutorial](https://spapros.readthedocs.io/en/latest/tutorials/spapros_tutorial_advanced_selection.html).